In [1]:
import pandas as pd
import json
import os
import pdfplumber
import numpy as np
from datetime import datetime

with open('DICCIONARIO_CODIGO_NOMBRE_FARMACOS.json', 'r', encoding = 'utf-8') as f:
    DICCIONARIO_CODIGO_NOMBRE_FARMACOS = json.load(f)

In [26]:
class Formateador():
    def __init__(self):
        pass

    def hacer_tabla_global(self):
        todas_las_entradas = self.obtener_entradas_todos_los_pacientes()
        columnas = ['Ingreso', 'Tipo muestra', 'Nº de Cultivo', 'Nº de Orden', 'Rut', 'Nombre', 'Servicio', 'Fecha Firma'] + list(DICCIONARIO_CODIGO_NOMBRE_FARMACOS.values())

        df = pd.DataFrame(todas_las_entradas, columns = columnas)
        return df

    def obtener_entradas_todos_los_pacientes(self):
        entradas_todos_los_pacientes = []

        for nombre_archivo in os.listdir():
            if nombre_archivo[-4:len(nombre_archivo)] == '.xls':
                print(nombre_archivo)
                entradas_de_un_paciente = self.obtener_entradas_de_un_paciente(nombre_archivo)
                for entrada_de_un_paciente in entradas_de_un_paciente:
                    entradas_todos_los_pacientes.append(entrada_de_un_paciente)

        return entradas_todos_los_pacientes

    def obtener_entradas_de_un_paciente(self, nombre_archivo):
        entradas = []

        datos_persona = self.obtener_datos_demograficos_de_un_paciente(nombre_archivo)
        lista_antibiogramas_persona = self.obtener_antibiogramas_de_un_paciente(nombre_archivo)
        lista_contaminados_persona = self.obtener_contaminados_de_un_paciente(nombre_archivo)
        lista_hongos_persona = self.obtener_cultivos_de_hongos_de_un_paciente(nombre_archivo)

        for antibiograma in lista_antibiogramas_persona:
            entrada_antibiograma = datos_persona + antibiograma
            entradas.append(entrada_antibiograma)
        
        for contaminados in lista_contaminados_persona:
            entrada_contaminados = datos_persona + contaminados
            entradas.append(entrada_contaminados)
        
        for hongos in lista_hongos_persona:
            entrada_hongos = datos_persona + hongos
            entradas.append(entrada_hongos)
        
        return entradas

    
    def obtener_antibiogramas_de_un_paciente(self, nombre_archivo):
        datos_totales = pd.read_excel(nombre_archivo)
        
        indice_fila_inicio_antibio, indice_fila_termino_antibio = self.identificar_localizacion_antibiograma(datos_totales)
        antibiograma_crudo = datos_totales.iloc[indice_fila_inicio_antibio: indice_fila_termino_antibio, :-2]
        antibiograma_formateado = self.formatear_tabla_antibiograma(antibiograma_crudo)
        lista_cepas_formato_df = self.separar_cepas(antibiograma_formateado)
        lista_cepas_formato_listas = list(map(self.mappear_resultados_a_formato_excel, lista_cepas_formato_df))

        return lista_cepas_formato_listas

    def identificar_localizacion_antibiograma(self, datos_totales):
        se_encontro_antibiograma = False
        columna_a_iterar = datos_totales.iloc[:, 0].fillna('-')
        for identificadores in enumerate(columna_a_iterar):
            if 'ANTIBIOGRAMA' in identificadores:
                indice_inicial = identificadores[0]
                se_encontro_antibiograma = True

            if (identificadores[1] == '-') and (se_encontro_antibiograma):
                indice_final = identificadores[0]

        return indice_inicial + 1, indice_final


    def formatear_tabla_antibiograma(self, antibiograma_crudo):
        antibiograma_formateado = antibiograma_crudo.copy()

        headers = ['ANTIBIOTICO']
        self.cantidad_de_cepas = int((len(antibiograma_formateado.columns) - 1)/ 2)
        for i in range(self.cantidad_de_cepas):
            headers.append(f'Cepa')
            headers.append(f'CIM')

        antibiograma_formateado.columns = headers
        antibiograma_formateado.ANTIBIOTICO = antibiograma_formateado.ANTIBIOTICO.map(DICCIONARIO_CODIGO_NOMBRE_FARMACOS)
        antibiograma_formateado.set_index(antibiograma_formateado.ANTIBIOTICO, inplace = True)
        antibiograma_formateado.drop(columns = 'ANTIBIOTICO', inplace = True)

        return antibiograma_formateado
    
    def separar_cepas(self, antibiograma_formateado):
        tablas_cepas = []
        for i in range(self.cantidad_de_cepas):
            df = antibiograma_formateado.iloc[:, i * 2: (i * 2) + 2]
            df = df[df['Cepa'].notna()]
            tablas_cepas.append(df)
        
        return tablas_cepas
    
    def mappear_resultados_a_formato_excel(self, tabla_una_cepa):
        cambiador_nomenclatura_sensibilidades = {'Sensible': 'S', 'Resistente': 'R'}
        diccionario_sensibilidades_a_llenar = {farmaco: None for farmaco in DICCIONARIO_CODIGO_NOMBRE_FARMACOS.values()}
        for farmaco in tabla_una_cepa.index:
            resultado_sensibilidad = tabla_una_cepa['Cepa'][farmaco]
            resultado_cambiado = cambiador_nomenclatura_sensibilidades[resultado_sensibilidad]
            diccionario_sensibilidades_a_llenar[farmaco] = resultado_cambiado
        
        lista_sensibilidades_llenas = list(diccionario_sensibilidades_a_llenar.values())
        return lista_sensibilidades_llenas 


    def obtener_contaminados_de_un_paciente(self, nombre_archivo):
        contaminados = []

        return contaminados

    def obtener_cultivos_de_hongos_de_un_paciente(self, nombre_archivo):
        hongos = []

        return hongos


    def obtener_datos_demograficos_de_un_paciente(self, nombre_archivo):
        nombre_archivo = nombre_archivo[:-4] + '.pdf'

        with pdfplumber.open(nombre_archivo) as pdf:
            datos_personales_relevantes = pdf.pages[0].extract_text().split('\n')[3:12]

            nombre_paciente = datos_personales_relevantes[0].split(':')[1][:-10]
            n_orden = datos_personales_relevantes[0].split(':')[-1]
            rut = datos_personales_relevantes[1].split(':')[-1]

            linea_ingreso = datos_personales_relevantes[4].split(' ')
            try:
                fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d-%m-%Y %H:%M:%S')
            except ValueError:
                fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d/%m/%Y %H:%M:%S')


            linea_firma = datos_personales_relevantes[5].split(' ')
            try:
                fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d-%m-%Y %H:%M:%S')
            except ValueError:
                fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d/%m/%Y %H:%M:%S')

            seccion = datos_personales_relevantes[6].split(':')[-1]
            tipo_muestra = datos_personales_relevantes[7].split(':')[-1]
            n_cultivo = datos_personales_relevantes[8].split(':', 1)[-1]

            return [fecha_ingreso, tipo_muestra, n_cultivo, n_orden, rut, nombre_paciente, seccion, fecha_firma]


In [27]:
formateador = Formateador()
lista = formateador.hacer_tabla_global()

942658_MARTINA_ANGELA_DIAZ_SASSO.xls
17 33
943532_NORMA_LOPEZ_LEYTON.xls
10 25
943594_MARIA_ARREPOL_RIFFO.xls
15 26
943596_MARIA_ARREPOL_RIFFO.xls


UnboundLocalError: local variable 'indice_inicial' referenced before assignment